![title](logo.png)

____
# Desafio 5 - Redução de Dimensionalidade e Seleção de Variáveis
___

### Base de Dados

Utilizaremos o _data set_ **Fifa 2019**, contendo originalmente 89 variáveis de mais de 18 mil jogadores do _game_ FIFA 2019.

Disponível em:https://www.kaggle.com/karangadiya/fifa19

### Objetivo

O objetivo deste desafio é explorar sobre como funciona o PCA e como podemos obter _data sets_ de dimensões mais baixas através dele.

> Obs.: Por favor, não modifique o nome das funções de resposta.



Neste desafios vamos explorar:
- Redução de dimensionalidade
- PCA
- Seleção de variáveis
- RFE
_______

## _Setup_ geral

In [92]:
# importa as bibliotecas
from math import sqrt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sct
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats as st
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [4]:
# algumas configurações para o matplotlib.
#%matplotlib inline

#from IPython.core.pylabtools import figsize


#figsize(12, 8)

#sns.set()

In [5]:
# carrega o dataframe
fifa = pd.read_csv("fifa.csv")

In [6]:
columns_to_drop = ["Unnamed: 0", "ID", "Name", "Photo", "Nationality", "Flag",
                   "Club", "Club Logo", "Value", "Wage", "Special", "Preferred Foot",
                   "International Reputation", "Weak Foot", "Skill Moves", "Work Rate",
                   "Body Type", "Real Face", "Position", "Jersey Number", "Joined",
                   "Loaned From", "Contract Valid Until", "Height", "Weight", "LS",
                   "ST", "RS", "LW", "LF", "CF", "RF", "RW", "LAM", "CAM", "RAM", "LM",
                   "LCM", "CM", "RCM", "RM", "LWB", "LDM", "CDM", "RDM", "RWB", "LB", "LCB",
                   "CB", "RCB", "RB", "Release Clause"
]

try:
    fifa.drop(columns_to_drop, axis=1, inplace=True)
except KeyError:
    logger.warning(f"Columns already dropped")

_______
## Inicia sua análise a partir daqui

In [7]:
# visualiza o dataset
fifa.head()

,Age,Overall,Potential,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
0,31,94,94,84.0,95.0,70.0,90.0,86.0,97.0,93.0,...,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0
1,33,94,94,84.0,94.0,89.0,81.0,87.0,88.0,81.0,...,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,26,92,93,79.0,87.0,62.0,84.0,84.0,96.0,88.0,...,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0
3,27,91,93,17.0,13.0,21.0,50.0,13.0,18.0,21.0,...,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0
4,27,91,92,93.0,82.0,55.0,92.0,82.0,86.0,85.0,...,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0


In [22]:
# remove os valores ausentes
fifa.dropna(inplace=True)

## Questão 1

Qual fração da variância consegue ser explicada pelo primeiro componente principal de `fifa`? Responda como um único float (entre 0 e 1) arredondado para três casas decimais.

In [41]:
def q1():
    pca = PCA().fit(fifa)
    evr = pca.explained_variance_ratio_
    return float(round(evr[0],3))  # pega o primeiro item do array gerado em evr

In [42]:
q1()

0.565

## Questão 2

Quantos componentes principais precisamos para explicar 95% da variância total? Responda como un único escalar inteiro.

In [103]:
def q2():
    pca_095 = PCA(n_components=0.95).fit(fifa)  # fit: encaixa o modelo para o dataset fifa
    components = pca_095.n_components_  # retorna o número de componentes
    return int(components)

In [104]:
q2()

15

In [57]:
# outra forma de encontrar os componentes do PCA é usando o sklearn que também nos permite achar diretamente as componentes principais
pca_095 = PCA(n_components=0.95)
X_reduced = pca_095.fit_transform(fifa)  # fit_transform: ajusta o modelo para o dataset e aplica a redução de dimensionalidade no dataset.

X_reduced.shape # segundo elemento da tupla é o número de componentes encontrados.

(18159, 15)

## Questão 3

Qual são as coordenadas (primeiro e segundo componentes principais) do ponto `x` abaixo? O vetor abaixo já está centralizado. Cuidado para __não__ centralizar o vetor novamente (por exemplo, invocando `PCA.transform()` nele). Responda como uma tupla de float arredondados para três casas decimais.

In [86]:
x = [0.87747123,  -1.24990363,  -1.3191255, -36.7341814,
     -35.55091139, -37.29814417, -28.68671182, -30.90902583,
     -42.37100061, -32.17082438, -28.86315326, -22.71193348,
     -38.36945867, -20.61407566, -22.72696734, -25.50360703,
     2.16339005, -27.96657305, -33.46004736,  -5.08943224,
     -30.21994603,   3.68803348, -36.10997302, -30.86899058,
     -22.69827634, -37.95847789, -22.40090313, -30.54859849,
     -26.64827358, -19.28162344, -34.69783578, -34.6614351,
     48.38377664,  47.60840355,  45.76793876,  44.61110193,
     49.28911284
]

In [87]:
def q3():
    pca = PCA(n_components=2).fit(fifa)
    x_vector = pca.components_.dot(x).round(3)  # dot: produto escalar de duas matrizes
    return tuple(x_vector)

In [88]:
q3()

(186.556, -6.592)

## Questão 4

Realiza RFE com estimador de regressão linear para selecionar cinco variáveis, eliminando uma a uma. Quais são as variáveis selecionadas? Responda como uma lista de nomes de variáveis.

O objetivo da **eliminação recursiva de recursos (RFE)** é selecionar recursos considerando recursivamente, considerando conjuntos cada vez menores de recursos.

In [101]:
def q4():
    X = fifa.drop(columns='Overall')  # retira a variável alvo do conjunto de features
    y = fifa['Overall']  # variável alvo(target)
    rfe_selector = RFE(estimator=LinearRegression(), n_features_to_select=5).fit(X, y)  # RFE usando a regressão linear como estimador
    selected_features = rfe_selector.get_support(indices=True)  #  retorna o array de indices das features 
    return list(X.columns[selected_features])  # retorna as features escolhidas

In [102]:
q4()

['Age', 'Potential', 'BallControl', 'Reactions', 'GKReflexes']

_________
## Conclusão
________

Em resumo, nesse desafio aprendi que: 

**Redução de dimensionalidade** é uma etapa necessária em modelos preditivos, pois muitos atributos podem confundir o classificador/regressor, o que torna a noção de distancia entre pontos do espaço inútil. O grande número de dimensões dos conjuntos de dados pode causar perdas no desempenho dos algoritmos. 

**PCA** é a Análise de Componentes Principais é um processo de transformação ortogonal para converter um conjunto de dados de variáveis possivelmente correlacionadas num conjunto de valores de variáveis linearmente não correlacionadas, que é dado o nome de componentes principais.  

**Seleção de variáveis** dentre todas as variáveis explicativas disponíveis, devemos encontrar um subconjunto de variáveis importantes para o modelo. Obtendo o máximo de informação por meio de um modelo com tantas variáveis independentes possíveis. E diminuir a variância da estimativa e o custo da coleta por meio de um modelo com menor número possível de variáveis. 

**RFE** o objetivo da eliminação recursiva de recursos é selecionar recursos considerando recursivamente, considerando conjuntos cada vez menores de recursos.Em que é atribuido um estimador de aprendizado supervisionado. 


________
Fontes de Pesquisa:

- http://www.portalaction.com.br/analise-de-regressao/27-selecao-de-variaveis
- https://www.ic.unicamp.br/~wainer/cursos/1s2017/ml/aula3.pdf
- http://ic.ufabc.edu.br/II_SIC_UFABC/resumos/paper_5_151.pdf
- https://www.it-swarm-pt.tech/pt/python/analise-de-componentes-principal-pca-em-python/1068857538/
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html
_____